[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

**ADVERTENCIA:**  

Para poder realizar exitosamente los ejercicios de esta notebook, es necesario haber seguido al pie de la letra y en orden sucesivo las instrucciones de todas las notebooks previas.

In [ ]:
%cd tutorial

El archivo *src/15/urls.py* contiene lo siguiente:

``` python 
from django.urls import path, re_path
from . import views, endpoint_views

urlpatterns = [path('', views.vista),
               path('carga', views.carga),
               re_path(r'^(?P<clave>[0-9]{4}$)', endpoint_views.clave),]
```

In [ ]:
%cp ../src/15/urls.py api/urls.py

``` python
from . import models
from django.views.decorators.csrf import csrf_exempt
from django.http import JsonResponse, HttpResponseNotFound, HttpResponseBadRequest

campos = {'numero_de_cuenta':(int, True), 'nombre':(str, True), 'primer_apellido':(str, True),
          'segundo_apellido':(str, False), 'carrera':(str, True), 'semestre':(int, True),
          'promedio':(float, True), 'al_corriente':(bool, True)}

carreras = ('Sistemas', 'Derecho', 'Actuaría', 'Arquitectura', 'Administración')

estructura_base = set(campos)


def reglas(valor, campo):
    if campo == "carrera" and valor not in carreras:
        return False
    elif campo == "semestre" and valor < 1:
        return False
    elif campo == "promedio" and (valor < 0 or valor > 10):
        return False
    elif (campo in ("nombre", "primer_apellido") and (valor == "")):
        return False
    else:
        return True         

    
def valida(dato, campo):
    tipo = campos[campo][0]
    try:
        if tipo is not str:
            valor = eval(dato)
        else:
            valor = dato
        if type(valor) is tipo or (tipo is float and type(dato) is int):
            return reglas(valor, campo)
        else:
            return False
    except:
        return False


@csrf_exempt
def clave(request, clave):
    '''Función de vista que define un endpoint correspondiente a una clave de 4 dígitos.
    Opera con los métodos GET, POST, DELETE.'''
    # Cuando la petición es GET va a obtener los datos del alumno con la clave correrspondiente.
    # Esta operación se realiza en caso de que exista un objeto con el número de cuenta.
    if request.method == "GET":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave) 
            return JsonResponse({campo:getattr(alumno, campo) for campo in campos})
        # La excepción models.Alumno.DoesNotExist se desencadena cuando la búsqueda no arroje un resultado.
        except models.Alumno.DoesNotExist:
            return HttpResponseNotFound()
        
    # Cuando la petición es DELETE el alumno es eliminado de la base de datos.
    # Esta operación se realiza en caso de que exista un objeto con el número de cuenta.
    if request.method == "DELETE":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave)
            alumno.delete()
            return JsonResponse({'estado': 'eliminado'})   
        except models.Alumno.DoesNotExist:
            return HttpResponseNotFound()
    # Cuando la petición es POST va a dar de alta los datos del alumno con la clave correspondiente y los datos enviados.
    # Esta operación se realiza en caso de que no exista un objeto con el número de cuenta.
    if request.method == "POST":
        try:
            alumno = models.Alumno.objects.get(numero_de_cuenta=clave) 
            return HttpResponseBadRequest()
        except models.Alumno.DoesNotExist:
            registro = request.POST.dict()
            registro["numero_de_cuenta"] = clave
            objeto = models.Alumno()
            estructura_registro = set(registro)
            if estructura_registro.issubset(estructura_base):
                for campo in estructura_base:
                    if valida(registro[campo], campo):
                        if campos[campo][0] is not str:
                            valor = eval(registro[campo])
                        else:
                            valor = registro[campo]
                        setattr(objeto, campo, valor)
                    else:
                        return HttpResponseBadRequest()
                objeto.save()
                return JsonResponse(registro)
            else:
                return HttpResponseBadRequest()
```


In [ ]:
%cp ../src/15/endpoint_views.py api/endpoint_views.py

In [ ]:
%cat api/endpoint_views.py

**ADVERTENCIAS:** 

* Al ejecutar la siguiente celda el servidor se inciará desde la notebook, por lo que para ejecutar cualquier otra celda es necesario interrumpir la ejecución del kernel.

* Asegúrese que no haya otro servicio escuchando en el puerto *5000*.

In [ ]:
!./manage.py runserver 0.0.0.0:5000

http://localhost:5000/api/1221

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>